In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import razor.flow as rf
from razor.flow import Pipeline
from razor.api import project_space_path
import typing as t

from razor.marketplace.blocks.rzt.infer_flow import DLInferBlock

from typing import Any

### LoadImages Block

In [8]:
@rf.block(executor=rf.ContainerExecutor(cores=1, memory=10000))
class LoadImages:
    path: str
    no_of_examples: int
    data: rf.SeriesOutput[dict]
    
    from razor.api import project_space_path
    def run(self):
        self.logger.debug('Running Load Images')
        
        npy = np.load(project_space_path(self.path))
            
        for image, label in zip(npy['images'][:self.no_of_examples], npy['labels'][:self.no_of_examples]):
            image = image.astype('float32') / 255.0
            label = np.eye(10)[[label]].reshape(-1)
            image = np.reshape(image, (image.shape[0], image.shape[1], 1))
            
            self.data.put({"images":image, "labels":label})
            
        self.logger.debug('Completed Load Images')

### Model

In [9]:
from razor.api import dlmodels
model = dlmodels.import_model(model_name='model_a', display=True)

INFO:rztdl.dl.model:Input Layer 'images'  validated successfully
INFO:rztdl.dl.model:BatchNormalization Layer 'BatchNormalization__1' validated successfully
INFO:rztdl.dl.model:Conv2D Layer 'Conv2D__1' validated successfully
INFO:rztdl.dl.model:MaxPool2D Layer 'MaxPool2D__1' validated successfully
INFO:rztdl.dl.model:Dropout Layer 'Dropout__1' validated successfully
INFO:rztdl.dl.model:Conv2D Layer 'Conv2D__2' validated successfully
INFO:rztdl.dl.model:MaxPool2D Layer 'MaxPool2D__2' validated successfully
INFO:rztdl.dl.model:Dropout Layer 'Dropout__2' validated successfully
INFO:rztdl.dl.model:Flatten Operator 'Flatten__1' validated successfully
INFO:rztdl.dl.model:Dense Layer 'Dense__1' validated successfully
INFO:rztdl.dl.model:Dropout Layer 'Dropout__3' validated successfully
INFO:rztdl.dl.model:Dense Layer 'Dense__2' validated successfully
INFO:rztdl.dl.model:BatchNormalization Layer 'BatchNormalization__2' validated successfully
INFO:rztdl.dl.model:Dense Layer 'Dense__3' validated

import rztdl.dl
from rztdl.dl.constants.string_constants import *
from rztdl.dl.helpers import *
from rztdl.dl.helpers.dtypes import DataTypes
from rztdl.dl.groups import Group
from rztdl.dl.components.layers import *
from rztdl.dl.hub import *
from rztdl.dl.components.metrics import *
from rztdl.dl.components.operators import *
from rztdl.dl.components.losses import *
from rztdl.dl.components.layers.recurrent.cells import *
from rztdl.dl.components.optimizers import *

# CustomCodeTemplates



model = rztdl.dl.RZTModel(name = 'model_a')
model.add(Input(dtype=DataTypes.FLOAT32,validate_shape=True,shape=[28, 28, 1],name='images',outputs='images_outputs'))
model.add(BatchNormalization(inputs='images_outputs',axis=[-1],momentum=0.99,epsilon=0.001,center=False,scale=False,moving_variance_initializer=Ones(),trainable=True,name='BatchNormalization__1',outputs='BatchNormalization__1_outputs'))
model.add(Conv2D(inputs='BatchNormalization__1_outputs',kernel_size=[4, 4],strides=[1, 1],padding=Pa

### SavePredictions Block

In [10]:
@rf.block(executor=rf.ContainerExecutor(cores=1, memory=10000))
class SavePreds:
    preds: rf.SeriesInput[Any]
    path: str
    import os
    import pandas as pd
    import numpy as np
    def run(self):
        y_true = []
        y_hats = []
        if os.path.exists(project_space_path(self.path)):
            os.remove(project_space_path(self.path))
        
        for batch in self.preds:
            y_true.extend(list(np.argmax(batch['labels'], axis=1)))
            y_hats.extend(list(batch['argmax']))
        df = pd.DataFrame({'y_true':y_true, 'y_hats':y_hats}).to_csv(project_space_path(self.path), index=False, mode='a')

### Infer Pipeline

In [11]:
loader_infer = LoadImages(path='data_test.npz',
                          no_of_examples=10)

predictor = DLInferBlock(model=model,
                         data=loader_infer.data,
                         load_path='image_classification_s2',
                         layers=['argmax','labels'],
                         batch_size=64
                        )
predictor.executor = rf.ContainerExecutor(cores=4, memory=12000)

save_preds = SavePreds(path="results.csv",
                       preds=predictor.infer_output)
   

In [12]:
infer_pipeline = rf.Pipeline(name ='infer_check', targets=[save_preds])

In [13]:
deployed_infer_pipeline = razor.api.engines("DS-Engine-Load-Test").execute(infer_pipeline)